In [2]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import pickle

# 파일 경로
file_path = '../DailyDoodle_ML/teset.xlsx'  # 절대 경로 사용

# 엑셀 파일 읽기
data = pd.read_excel(file_path, engine='openpyxl')

# 데이터의 첫 몇 줄을 확인
print(data.head())

# 텍스트 데이터와 감정 레이블 추출
text_data = data['사람문장1'].dropna()  # 결측값 제거
emotion_labels = data['감정_대분류'][text_data.index]

# 감정 레이블 인코딩
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(emotion_labels)

# 텍스트 데이터 벡터화 (TF-IDF)
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(text_data).toarray()

# 학습 데이터와 테스트 데이터로 분할
X_train, X_test, y_train, y_test = train_test_split(X, encoded_labels, test_size=0.2, random_state=42)

# 로지스틱 회귀 모델 학습
model = LogisticRegression()
model.fit(X_train, y_train)

# 모델 평가
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

# 모델과 벡터라이저 저장
with open('sentiment_model.pkl', 'wb') as model_file:
    pickle.dump(model, model_file)

with open('vectorizer.pkl', 'wb') as vectorizer_file:
    pickle.dump(vectorizer, vectorizer_file)

with open('label_encoder.pkl', 'wb') as label_encoder_file:
    pickle.dump(label_encoder, label_encoder_file)


   Unnamed: 0  연령  성별     상황키워드  신체질환 감정_대분류 감정_소분류  \
0           1  청년  여성  진로,취업,직장  해당없음     분노  노여워하는   
1           2  청년  여성  진로,취업,직장  해당없음     분노  노여워하는   
2           3  청년  여성  진로,취업,직장  해당없음     분노  노여워하는   
3           4  청년  여성  진로,취업,직장  해당없음     분노  노여워하는   
4           5  청년  여성  진로,취업,직장  해당없음     분노  노여워하는   

                                               사람문장1  \
0                          일은 왜 해도 해도 끝이 없을까? 화가 난다.   
1     이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.   
2  회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...   
3  직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...   
4              얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.   

                                              시스템문장1  \
0                        많이 힘드시겠어요. 주위에 의논할 상대가 있나요?   
1           급여가 줄어 속상하시겠어요. 월급이 줄어든 것을 어떻게 보완하실 건가요?   
2  회사 동료 때문에 스트레스를 많이 받는 것 같아요. 문제 해결을 위해 어떤 노력을 ...   
3  관련 없는 심부름을 모두 하게 되어서 노여우시군요. 어떤 것이 상황을 나아질 수 있...   
4  무시하는 것 같은 태도에 화가 나셨군요. 상대방의 어떤 행동이 그런 감정을 유발하는...

c:\python312\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [5]:
import pickle
import numpy as np

# 저장된 모델, 벡터라이저, 레이블 인코더 로드
with open('sentiment_model.pkl', 'rb') as model_file:
    model = pickle.load(model_file)

with open('vectorizer.pkl', 'rb') as vectorizer_file:
    vectorizer = pickle.load(vectorizer_file)

with open('label_encoder.pkl', 'rb') as label_encoder_file:
    label_encoder = pickle.load(label_encoder_file)

# 사용자 입력 받기
user_input = input("감정을 분석할 문장을 입력하세요: ")

# 입력된 텍스트를 벡터화
user_input_vectorized = vectorizer.transform([user_input]).toarray()

# 감정 예측
predicted_emotion_index = model.predict(user_input_vectorized)
predicted_emotion = label_encoder.inverse_transform(predicted_emotion_index)

print(f"예측된 감정: {predicted_emotion[0]}")


예측된 감정: 슬픔
